In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from pprint import pprint


X = pd.read_csv("mushrooms.csv")
for column in X.columns: 
        X[column] = pd.factorize(X[column])[0]

# Help functions

In [8]:
def train_test_split(X, test_size):

    
    #sets the test size if the parameter is a percentage
    if isinstance(test_size, float):
        test_size = round(test_size * len(X))
    
    #test_indices is picked randomly
    indices = X.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_set = X.loc[test_indices]
    train_set = X.drop(test_indices)
    
    return train_set, test_set

In [9]:
#same numbers of data will appear every time
random.seed(0)
#gets training_set and test_set of the dataframe X
train_X, test_X = train_test_split(X, test_size=0.2)

In [10]:
#sets data to a two dimmensional numpy array
data = train_X.values

In [11]:
#takes in X.values and checks on its labels, returns true or false if there is only one class
def check_purity(data):
    """
    Title: 
        check_purity
    
    Description: 
        Check if a dataset only contains one label
    
    Parameters:
        data(array): a dataset of the dataframe
    
    Returns:
        True/False: depending on the length of the labels
    """
    
    y = data[:, 0]
    unique_classes = np.unique(y)
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [24]:
def classify_data(data):
    """
    Title: 
        classify_data
    
    Description: 
        Classifies the data. 
    
    Parameters:
        data(array): a dataset of the dataframe
    
    Returns:
        classification(int): classification of the dataset. 0 or 1
    """
    y = data[:, 0]
    unique_classes, counts_unique_classes = np.unique(y, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [25]:
def get_potential_splits(data):
    
    """
    Title: 
        get_potential_splits
    
    Description: 
        Splits the dataset into a dictionary that contains the 
        potential splits of a column based on the values
    
    Parameters:
        data(array): a dataset of the dataframe
    Returns:
        potential_splits(dict) : key = column_index and values is potential split values
    """
    
    potential_splits = {}
    _, n_columns = data.shape
    
    #exclude the label which is the first column
    for column_index in range(1, n_columns): 
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [26]:
def split_data(data, split_column, split_value):
    
    """
    Title:
        split_data
    
    Desciption:
        splits the dataset in two based on column and value
        
    Parameters:
        data(array): dataset of the dataframe
        split_column(int): the column index of best splitting column
        split_value(float): the value of where to split at
    
    Returns
        data_below(array): a dataset
        data_above(array): a dataset
    
    """
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

In [27]:
def entropy(data):
    
    """
    Title:
        entropy
        
    Description:
        calculates the entropy of a dataset
        
    Parameters:
        data(array): a dataset
        
    Returns:
        entropy(float): entropy of a dataset
    """
    
    y = data[:, 0]
    _, counts = np.unique(y, return_counts=True)

    p = counts / counts.sum()
    entropy = sum(p * -np.log2(p))
     
    return entropy

In [28]:
def overall_entropy_split(data_below, data_above):
    
    """
    Title:
        overall_entropy_split
        
    Description:
        Calculates the overall entropy of two datasets
        
    Parameters:
        data_below(array): a dataset
        data_above(array): a dataset
        
    Returns:
        overall_entropy(float): the overall entropy of two datasets
    """
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * entropy(data_below) 
                      + p_data_above * entropy(data_above))
    
    return overall_entropy

In [29]:
def gini(data):
 
    
    y = data[:, 0]
    #counts how many times a label occurs, lies the sum of each label in the array counts
    _, counts = np.unique(y, return_counts=True)
    prob = counts / counts.sum()
    gini = 1 - sum(prob**2)
    return gini

In [30]:
def overall_gini_split(data_below, data_above):
    
    """
    Title:
        overall_gini_split
        
    Description:
        calculates the overall gini index of two datasets
        
    Parameters:
        data_below(array): a dataset
        data_above(array): a dataset
        
    Returns:
        overall_gini(float): the overall gini index of two datasets
    """
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    
    overall_gini = (p_data_below * gini(data_below)
                   + p_data_above * gini(data_above))
    
    return overall_gini

In [31]:
def determine_best_split(data, potential_splits, impurity_measure):
    
    """
    Title:
        determine_best_split
        
    Description:
        Finds the best split based on the lowest entropy or gini index
        
    Parameters:
        data(array): a dataset
        potential_splits(dict): key = column_index and values is potential split values
        impurity_measure(str): a string that determines if best split
                                is calculated with gini index or entropy
        
    Returns:
        best_split_column(int): index of best column to split at
        best_split_value(float): the best value to split the column at
    """
    
    overall_entropy = 9999
    overall_gini = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            
            if (impurity_measure == 'gini'):
                current_overall_gini = overall_gini_split(data_below, data_above)
                if current_overall_gini <= overall_gini:
                    overall_gini = current_overall_gini
                    best_split_column = column_index
                    best_split_value = value
                    
            else:
                current_overall_entropy = overall_entropy_split(data_below, data_above)
                if current_overall_entropy <= overall_entropy:
                    overall_entropy = current_overall_entropy
                    best_split_column = column_index
                    best_split_value = value
    
    return best_split_column, best_split_value

# Learn function

In [32]:
def learn(X,impurity_measure, counter=0, min_samples=2, max_depth=5):
    
    """
    Title:
        learn
        
    Description:
        builds the tree recursivley
        
    Parameters:
        X(dataframe): a dataframe read from a csv file
        impurity_measure(str): a string that determines if best split
                                is calculated with gini index or entropy
        counter(int): counts how many times the recursion has happend
        min_samples(int): used to check that the length of the data is above or equeal a sertain number
        max_depth(int): the maximum depth the tree can have
        
    Returns:
        classification(int): classification of the dataset. 0 or 1
        sub_tree(dict): keys = question values = list of answer
    """
    
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = X.columns
        data = X.values
    else:
        data = X           
    
    
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    #start of the recursion
    else:    
        counter += 1

        #calling the help functions to get split on the best column and value
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits, impurity_measure)
        data_below, data_above = split_data(data, split_column, split_value)
        
        #making the sub stree and the questions
        column_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(column_name, split_value)
        sub_tree = {question: []}
        
        #calling the learn method to get answers recursivley
        yes_answer = learn(data_below, impurity_measure, counter, min_samples, max_depth)
        no_answer = learn(data_above, impurity_measure, counter, min_samples, max_depth)
        
        #when the answers are the same there is no need for asking the question
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [55]:
tree = learn(train_X, impurity_measure='entropy', max_depth=5)
pprint(tree)

{'odor <= 3.5': [{'odor <= 0.5': [0,
                                  {'spore-print-color <= 3.5': [1,
                                                                {'stalk-root <= 3.0': [{'habitat <= 2.5': [0,
                                                                                                           1]},
                                                                                       {'gill-size <= 0.5': [0,
                                                                                                             1]}]}]}]},
                 0]}


In [56]:
#makes and example of row number 0
example = test_X.iloc[0]
example

class                          0
cap-shape                      3
cap-surface                    0
cap-color                      4
bruises                        1
odor                           7
gill-attachment                0
gill-spacing                   0
gill-size                      0
gill-color                     8
stalk-shape                    1
stalk-root                     4
stalk-surface-above-ring       2
stalk-surface-below-ring       3
stalk-color-above-ring         0
stalk-color-below-ring         0
veil-type                      0
veil-color                     0
ring-number                    0
ring-type                      1
spore-print-color              4
population                     3
habitat                        3
classification                 0
classification_correct      True
Name: 6917, dtype: object

In [57]:
#classifies an example, a row by running it through the decision tree
def classify_example(example, tree):
    
    """
    Title:
        classify_example
        
    Description:
        classifies an example, a row by running it through the decision tree recursivley
        
    Parameters:
        example(array): a row in a dataset
        tree(dict): keys = question values = list of answer
        
    Returns:
        answer(int): class of the example. 0 or 1
        clssify_example: recursion
    """
    
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [58]:
potential_splits = get_potential_splits(data)
split_column, split_value = determine_best_split(data,potential_splits, impurity_measure='entropy')
data_below, data_above = split_data(data, split_column, split_value)
print('split column ', split_column)
print('split value ', split_value)

split column  5
split value  3.5


In [59]:
classify_example(example, tree)

0

In [60]:
def score(X, tree):
    
    """
    Title:
        score
        
    Description:
        calculates the accuracy of the test set by running it through the tree
        
    Parameters:
        X(dataframe): a dataframe read from a file
        tree(dict): keys = question values
    Returns:
        accuracy(float): accuracy of the test set runned through the tree
    """

    X["classification"] = X.apply(classify_example, axis=1, args=(tree,))
    X["classification_correct"] = X["classification"] == X["class"]
    
    accuracy = X["classification_correct"].mean()
    
    return accuracy

In [61]:
accuracy = score(test_X, tree)
accuracy

0.9993846153846154

# Sklearn desicion tree

In [47]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X = pd.read_csv("mushrooms.csv")
for column in X.columns: 
        X[column] = pd.factorize(X[column])[0]

all_inputs = X[X.columns[1:]].values
all_classes = X['class'].values

#calling the train_test_split method and sets train_set = 0.2 of the dataset
(train_inputs, test_inputs, train_classes, test_classes) = train_test_split(all_inputs, all_classes, train_size=0.8, test_size=0.2,random_state=1)

dtc = DecisionTreeClassifier('entropy')
dtc.fit(train_inputs, train_classes)
dtc.score(test_inputs, test_classes)

1.0